Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [79]:
from collections import namedtuple
from random import choice
from tqdm.auto import tqdm
import numpy as np
from heapq import heappush, heappop
from icecream import ic


In [80]:
PUZZLE_DIM = 3
action = namedtuple('Action', ['pos1', 'pos2'])

In [81]:
COST = 0

def available_actions(state: np.ndarray) -> list['Action']:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions



def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    global COST
    COST += 1
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

In [82]:
RANDOMIZE_STEPS = 100_000
state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    state = do_action(state, choice(available_actions(state)))
state

Randomizing: 100%|██████████| 100000/100000 [00:00<00:00, 164257.12it/s]


array([[0, 2, 8],
       [5, 7, 1],
       [6, 3, 4]])

In [83]:
# Function to calculate the Manhattan distance relative to the map
def heuristic(state, goal_positions):
        dist = 0
        for i in range(PUZZLE_DIM):
            for j in range(PUZZLE_DIM):
                if state[i, j] != 0:
                    goal_x, goal_y = goal_positions[state[i, j]]
                    # Sum of vertical and horizontal distances between the tile and its goal position
                    dist += abs(i - goal_x) + abs(j - goal_y)
        return dist


def a_star(initial_state: np.ndarray, goal_state: np.ndarray):
    global COST 
    COST = 0
    # Create a map of the goal positions
    goal_positions = {value: divmod(idx, PUZZLE_DIM) for idx, value in enumerate(goal_state.flatten())}

    open_set = []  # Priority queue
    came_from = {}  # Tracks where states came from
    visited = set()  # Already visited states

    # Initialize the queue with the initial state
    heappush(open_set, (0, initial_state.tobytes()))  # (f, state)
    g_score = {initial_state.tobytes(): 0}  # Cost to reach the states

    while open_set:
        # Extract the state with the lowest score
        _, current_bytes = heappop(open_set)

        # Skip already visited states
        if current_bytes in visited:
            continue

        # Add the current state to the visited set
        visited.add(current_bytes)

        # Convert the byte array back to the current state
        current_state = np.frombuffer(current_bytes, dtype=int).reshape(PUZZLE_DIM, PUZZLE_DIM)

        # Check if the current state is the goal state
        if np.array_equal(current_state, goal_state):
            # Reconstruct the path
            path = []
            while current_bytes in came_from:
                current_bytes, action = came_from[current_bytes]
                path.append(action)
            return path[::-1], COST  # Reverse the path to get the solution

        # Explore the neighbors
        for action in available_actions(current_state):
            neighbor = do_action(current_state, action)
            neighbor_bytes = neighbor.tobytes()

            # Skip already visited states
            if neighbor_bytes in visited:
                continue

            # Calculate the new cost
            tentative_g_score = g_score[current_bytes] + 1

            # Update the data if a better path is found
            if tentative_g_score < g_score.get(neighbor_bytes, float('inf')):
                came_from[neighbor_bytes] = (current_bytes, action)
                g_score[neighbor_bytes] = tentative_g_score
                f_score = tentative_g_score + heuristic(neighbor, goal_positions)
                heappush(open_set, (f_score, neighbor_bytes))

    # Return None if no solution is found
    return None


def greedy_best_first_search(initial_state: np.ndarray, goal_state: np.ndarray, max_depth: int = 200) -> int:
    global COST
    COST = 0
    
    goal_positions = {value: divmod(idx, PUZZLE_DIM) for idx, value in enumerate(goal_state.flatten())}

    # Initialize the priority queue (open_set)
    open_set = []
    visited = set()  # Already visited states
    heappush(open_set, (heuristic(initial_state, goal_positions), 0, initial_state.tobytes()))  # (heuristic, depth, state)
    
    # Main loop
    while open_set:
        _, depth, current_bytes = heappop(open_set)

        if current_bytes in visited:
            continue
        visited.add(current_bytes)  # Add the current state to the visited set

        current_state = np.frombuffer(current_bytes, dtype=int).reshape(PUZZLE_DIM, PUZZLE_DIM)

        # If the goal state is reached, return the number of steps (depth)
        if np.array_equal(current_state, goal_state):
            return depth, current_state, COST

        # Check if we have exceeded the maximum depth
        if depth >= max_depth:
            continue

        # Explore the neighbors
        for action in available_actions(current_state):
            neighbor = do_action(current_state, action)
            neighbor_bytes = neighbor.tobytes()

            if neighbor_bytes not in visited:  # Explore only unvisited states
                # Add the neighbor to the queue with an incremented depth
                heappush(open_set, (heuristic(neighbor, goal_positions), depth + 1, neighbor_bytes))
            
    print("No solution found within the specified depth limit.")
    return -1  # Indicates that the solution was not found


# Initial state and goal state
initial_state_greedy = initial_state_a_star = state
goal_state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
state


array([[0, 2, 8],
       [5, 7, 1],
       [6, 3, 4]])

In [84]:
greedy_num_steps, current_state, cost = greedy_best_first_search(initial_state_greedy, goal_state)
print(f"Solution found in {greedy_num_steps} steps with a cost of: {cost} using Greedy Best-First Search")
current_state


Solution found in 48 steps with a cost of: 931 using Greedy Best-First Search


array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 0]])

In [85]:
solution_a_star, cost = a_star(initial_state_a_star, goal_state)
for s in solution_a_star: 
    initial_state_a_star = do_action(initial_state_a_star, s)

print(f"Solution found in {len(solution_a_star)} steps with a cost of: {cost} using A*")
initial_state_a_star


Solution found in 22 steps with a cost of: 429 using A*


array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 0]])